# Normalizing Flows

![Status](https://img.shields.io/static/v1.svg?label=Status&message=WIP&color=orange)

**Author:** Julius Cathalina

**Last Updated (dd-mm-yyyy):** 01-02-2025

In [1]:
## Standard libraries
import os
import math
import time
import numpy as np 

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

In [2]:
import torch
import torch.nn as nn
import lightning as L
import torch.nn.functional as F
try:
    import torchdiffeq
except ModuleNotFoundError:
    !pip install --quiet torchdiffeq
    import torchdiffeq


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [3]:
from typing import Callable

In [4]:
L.seed_everything(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Seed set to 42


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda
